In [1]:
import pandas as pd
import numpy as np
import os

import re

from functools import reduce

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
weekly_data_tables = os.listdir('weekly_runs/')
print(weekly_data_tables)

['week_1.csv', 'week_10.csv', 'week_11.csv', 'week_2.csv', 'week_3.csv', 'week_4.csv', 'week_5.csv', 'week_6.csv', 'week_7.csv', 'week_8.csv', 'week_9.csv']


In [20]:
collection = []

for weekly_data in weekly_data_tables:
    collection.append(pd.read_csv('weekly_runs/' + weekly_data))

In [21]:
%%time

pred_types = ['pred_rf', 'pred_gbm', 'pred_lgb', 'pred_catboost', 'pred_mean']
pred_types_collected = {}

for pred_type in pred_types:
    
    keep_cols = ['Team', 'season', 'week_marker', pred_type]    
    pred_types_data = []
    
    for i in collection:
        data_for_pred_type = i[keep_cols].copy()
        
        week_marker = data_for_pred_type['week_marker'].unique().item()
        data_for_pred_type.rename(columns = {pred_type : pred_type + '_' + week_marker}, inplace = True)
        data_for_pred_type.drop(['week_marker', 'season'], 1, inplace = True)
        
        pred_types_data.append(data_for_pred_type)
        
    pred_types_data = reduce(lambda  a,b: pd.merge(a,b, on = ['Team'], how='outer'), pred_types_data)
    pred_types_data.sort_values(pred_types_data.columns[-1], ascending = False, inplace = True)
    
    columns = [re.sub('[^0-9]', '', i) if i != 'Team' else i for i in pred_types_data.columns]
    pred_types_data.columns = columns
        
    pred_types_collected[pred_type] = pred_types_data  

Wall time: 719 ms


In [22]:
# pred_types_collected['pred_rf']
# pred_types_collected['pred_gbm']
# pred_types_collected['pred_lgb']
# pred_types_collected['pred_catboost']
# pred_types_collected['pred_mean']

In [23]:
select_pred_type = 'pred_mean'

pred_types_collected[select_pred_type].head()

,Team,1,10,11,2,3,4,5,6,7,8,9
0,Arizona Cardinals,0.512468,0.253320,0.259717,0.319694,0.332717,0.347861,0.325952,0.349341,0.385269,0.343536,0.332476
1,Los Angeles Rams,0.486665,0.233676,0.233676,0.434764,0.396305,0.345544,0.284640,0.329892,0.309442,0.318350,0.323857
5,Tampa Bay Buccaneers,0.295510,0.267443,0.249250,0.351935,0.355476,0.218915,0.296834,0.269347,0.259826,0.283714,0.283714
14,Dallas Cowboys,0.170148,0.297298,0.181038,0.088968,0.191692,0.237991,0.208854,0.280636,0.280636,0.284810,0.259580
20,Buffalo Bills,0.108006,0.278152,0.196157,0.278697,0.275407,0.288686,0.387259,0.331091,0.331091,0.308676,0.255495


In [27]:
last_week = str(max([int(i) for i in pred_types_collected[select_pred_type].columns[1:]]))

data_for_plotting = pred_types_collected[select_pred_type].sort_values(last_week, ascending = False).head(10).melt(id_vars = 'Team', var_name = 'week', value_name = 'predicted_probability')

In [30]:
data_for_plotting

,Team,week,predicted_probability
0,Arizona Cardinals,1,0.512468
2,Los Angeles Rams,1,0.486665
8,San Francisco 49ers,1,0.452701
1,Tampa Bay Buccaneers,1,0.295510
6,Las Vegas Raiders,1,0.193140
...,...,...,...
25,New England Patriots,11,0.163799
26,Las Vegas Raiders,11,0.151356
27,Green Bay Packers,11,0.133362
28,San Francisco 49ers,11,0.132661


In [29]:
data_for_plotting['week'] = data_for_plotting['week'].astype(int)
data_for_plotting.sort_values(['week', 'predicted_probability'], ascending = [True, False], inplace = True)

In [34]:
legend_order = data_for_plotting[data_for_plotting['week'] == int(last_week)]['Team'].tolist()

fig = px.line(data_for_plotting, 'week', 'predicted_probability', color = 'Team', 
              title = 'Top 10 teams to make it to the Super Bowl in the 2021 NFL season', 
              labels = {'week' : 'Regular season week', 'predicted_probability' : 'Predicted probability (making SB)'},
              width = 950, height = 475, category_orders={"Team": legend_order})
fig.update_yaxes(tickfont = {'size' : 8})
fig.update_traces(mode = 'markers+lines')
fig.show()

Largest differences to week 1

In [35]:
first_week = str(data_for_plotting['week'].min())
last_week = str(data_for_plotting['week'].max())

In [36]:
first_last_weeks = pred_types_collected[select_pred_type][['Team', first_week, last_week]]

first_last_weeks['diff_ABS'] = first_last_weeks[last_week] - first_last_weeks[first_week]
first_last_weeks['diff_REL'] = first_last_weeks[last_week] / first_last_weeks[first_week] - 1

In [37]:
first_last_weeks.sort_values('diff_ABS')[['Team', 'diff_ABS']]

,Team,diff_ABS
2,San Francisco 49ers,-0.320040
3,Seattle Seahawks,-0.296582
1,Los Angeles Rams,-0.252990
0,Arizona Cardinals,-0.252751
4,Philadelphia Eagles,-0.218425
6,New Orleans Saints,-0.200558
9,Houston Texans,-0.183621
7,Cincinnati Bengals,-0.166989
8,Kansas City Chiefs,-0.160519
11,Baltimore Ravens,-0.130342


In [38]:
first_last_weeks.sort_values('diff_REL')[['Team', 'diff_REL']]

,Team,diff_REL
9,Houston Texans,-0.780691
3,Seattle Seahawks,-0.770962
2,San Francisco 49ers,-0.706958
6,New Orleans Saints,-0.702169
4,Philadelphia Eagles,-0.688014
15,Pittsburgh Steelers,-0.685181
13,Cleveland Browns,-0.682545
16,Detroit Lions,-0.682195
8,Kansas City Chiefs,-0.652493
11,Baltimore Ravens,-0.646027
